In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 721.3/721.3 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 51.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 56.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 68.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

In [8]:
import xml.etree.ElementTree as ET
import os

def convert_voc_annotation(data_path, data_type, output_path, classes):
    img_ids = open(f'{data_path}/ImageSets/Main/{data_type}.txt').read().strip().split()
    with open(output_path, 'w') as file:
        for img_id in img_ids:
            file.write(f'{data_path}/JPEGImages/{img_id}.jpg')
            annotation_file = f'{data_path}/Annotations/{img_id}.xml'
            tree = ET.parse(annotation_file)
            root = tree.getroot()
            for obj in root.iter('object'):
                difficult = obj.find('difficult').text
                cls = obj.find('name').text
                if cls not in classes or int(difficult) == 1:
                    continue
                cls_id = classes.index(cls)
                xmlbox = obj.find('bndbox')
                b = (int(xmlbox.find('xmin').text), int(xmlbox.find('ymin').text), int(xmlbox.find('xmax').text), int(xmlbox.find('ymax').text))
                file.write(" " + ",".join([str(a) for a in b]) + ',' + str(cls_id))
            file.write('\n')
    print(f"Successfully converted {data_type} annotations.")

classes = ['person', 'bird', 'cat', 'cow', 'dog', 'horse', 'sheep', 'aeroplane', 'bicycle', 'boat', 'bus', 'car', 'motorbike', 'train', 'bottle', 'chair', 'dining table', 'potted plant', 'sofa', 'tv/monitor']  # Add all Pascal VOC classes
import os

output_directory = '/content/drive/My Drive/Stat542Midterm/StatisticalLearningProject/data/'
output_file = 'outputYOLO_train.txt'
output_path = os.path.join(output_directory, output_file)

# Create the directory if it doesn't exist
os.makedirs(output_directory, exist_ok=True)

# Now you can safely call the function without causing a FileNotFoundError
convert_voc_annotation('/content/drive/My Drive/Stat542Midterm/StatisticalLearningProject/data/VOCdevkit/VOC2012/', 'train', output_path, classes)


KeyboardInterrupt: 

In [ ]:
%%writefile dataset.yaml
train: /content/drive/My Drive/path_to_your_dataset/train/images
val: /content/drive/My Drive/path_to_your_dataset/val/images

nc: 20
names: ['aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus', 'car', 'cat', 'chair', 'cow', 'diningtable', 'dog', 'horse', 'motorbike', 'person', 'pottedplant', 'sheep', 'sofa', 'train', 'tvmonitor']

In [ ]:
!yolo task=detect mode=train model=yolov8.pt data=dataset.yaml epochs=50 batch=16 imgsz=640

In [ ]:
!yolo task=detect mode=predict model=runs/train/exp/weights/best.pt source=/path/to/your/image.jpg